In [1]:
from dataclasses import dataclass
import math

LAPSE_RATE_FT = -0.0000068756  # ratio R per foot $/ft$

REFERENCE_TEMP_degR = 518.67  # $\degree{R}$
REFERENCE_SEA_LEVEL_PRESSURE_SQFT = 2116
REFERENCE_SEA_LEVEL_DENSITY_SLUGS_FT3 = 0.002378

ATMOSPHERE_LIMITS = {
    "Troposphere": 36_089,
    "Lower Stratosphere": 65_671,
    "Middle Stratosphere": 104_987, 
    "Upper Stratosphere": 154_199, 
    "Stratopause": 167_323,
    "Lower Mesophere": 232_940,
    "Upper Mesophere": 278_386,
}

@dataclass
class Atmosphere:
    temp_ratio: float
    press_ratio: float
    density_ratio: float
    temperature: float
    pressure: float
    density: float

def atmos_property(altitude_ft: float) -> tuple:
    R = 0.0
    temp_ratio = 0.0
    press_ratio = 0.0
    density_ratio = 0.0

    # math.exp(x) is the same as e^x (e is Euler's number)

    if altitude_ft < ATMOSPHERE_LIMITS["Troposphere"]:
        R = (1 + LAPSE_RATE_FT * altitude_ft)
        temp_ratio = R
        press_ratio = R ** 5.2561
        density_ratio = R ** 4.2561
    elif altitude_ft >= ATMOSPHERE_LIMITS["Troposphere"] and altitude_ft < ATMOSPHERE_LIMITS["Lower Stratosphere"]:
        R = ((altitude_ft - ATMOSPHERE_LIMITS["Troposphere"]) * -1.0) / 20_806
        temp_ratio = 0.751865
        press_ratio = 0.223361 * math.exp(R)
        density_ratio = 0.297176 * math.exp(R)
    elif altitude_ft >= ATMOSPHERE_LIMITS["Lower Stratosphere"] and altitude_ft < ATMOSPHERE_LIMITS["Middle Stratosphere"]:
        temp_ratio = 0.682457 + altitude_ft / 945_374
        press_ratio = (0.988626 + altitude_ft / 652_600) ** -34.1632
        density_ratio = (0.978261 + altitude_ft / 659_515) ** -35.1632
    elif altitude_ft >= ATMOSPHERE_LIMITS["Middle Stratosphere"] and altitude_ft < ATMOSPHERE_LIMITS["Upper Stratosphere"]:
        temp_ratio = (0.482561 + altitude_ft) / 337_634
        press_ratio = (0.898309 + altitude_ft / 181_373) ** -12.20114
        density_ratio = (0.857003 + altitude_ft / 190_115) ** -13.20114
    elif altitude_ft >= ATMOSPHERE_LIMITS["Upper Stratosphere"] and altitude_ft < ATMOSPHERE_LIMITS["Stratopause"]:
        R = ((altitude_ft - ATMOSPHERE_LIMITS["Upper Stratosphere"]) * -1) / 25_992
        temp_ratio = 0.939268
        press_ratio = 0.00109456 ** math.exp(R)
        density_ratio = 0.00116533 ** math.exp(R)
    elif altitude_ft >= ATMOSPHERE_LIMITS["Stratopause"] and altitude_ft < ATMOSPHERE_LIMITS["Lower Mesophere"]:
        temp_ratio = 1.434843 - altitude_ft / 337_634
        press_ratio = (0.838263 - altitude_ft / 577_922) ** 12.20114
        density_ratio = (0.79899 - altitude_ft / 606_330) ** 11.20114
    elif altitude_ft >= ATMOSPHERE_LIMITS["Lower Mesophere"] and altitude_ft < ATMOSPHERE_LIMITS["Upper Mesophere"]:
        temp_ratio = 1.237723 - altitude_ft / 472_687
        press_ratio = (0.917131 - altitude_ft / 637_919) ** 17.0816
        density_ratio = (0.900194 - altitude_ft / 649_922) ** 16.0816
    else:
        raise ValueError("altitude_ft not within known bounds (0-278,386ft)")

    temp_degR = temp_ratio * REFERENCE_TEMP_degR
    press_slugft = press_ratio * REFERENCE_SEA_LEVEL_PRESSURE_SQFT
    density_slugft = density_ratio * REFERENCE_SEA_LEVEL_DENSITY_SLUGS_FT3

    # temperature ratio, pressure ratio, density ratio, temperature, pressure, density
    return Atmosphere(temp_ratio, press_ratio, density_ratio, temp_degR, press_slugft, density_slugft)

atmos_property(8500)

Atmosphere(temp_ratio=0.9415574, press_ratio=0.7286788606898387, density_ratio=0.7739080598695721, temperature=488.35757665799997, pressure=1541.8844692196985, density=0.0018403533663698423)

In [4]:
def eq19_2_14_max_level_airspeed_prop(
    s: float, k: float, cd_min: float, w: float, h: float, bhp: float, eta: float
) -> float:
    """
    Determines maximum speed of a propeller powered aircraft.
    Solves equation 19.27 using the Bisection method.

    s: wing area, ft^2
    k: lift-induced drag constant
    cd_min: minimum drag coefficient
    w: weight, lbf
    h: altitude, ft
    bhp: piston engine power in BHP
    eta: propeller efficiency

    Returns: airspeed in ft/s
    """

    # could be replaced with equation 16.18
    rho = atmos_property(h).density

    v0 = 0
    v1 = 500
    f0 = -1100 * eta * bhp

    v_mid = 0.0
    f_mid = 0.0

    for _ in range(100):
        # compute midpoint values
        v_mid = 0.5 * (v0 + v1)
        f_mid = eq19_27_f_of_v(rho, s, cd_min, k, w, bhp, eta, v_mid)

        if f0 * f_mid < 0:
            v1 = v_mid
        else:
            v0 = v_mid
            f0 = v_mid
        
        # evaluate difference and adjust vini for next iteration
        if abs(v1 - v0) < 0.0001:
            flag = 0
            return 0.5 * (v0 + v1)

    return -1


def eq19_27_f_of_v(rho: float, s: float, cd_min: float, k: float, w: float, bhp: float, eta: float, v: float) -> float:
    """
    Calculates the value of equation 19.27
    """
    k1 = math.pow(550 * eta * bhp, 2) - math.pow(2 * w * v, 2) * cd_min * k

    # this is a trick to ensure the routine can solve to higher altitudes
    if k1 < 0:
        k1 = 0

    # calcuate the value of equation 19.27
    return (rho * s * cd_min * math.pow(v, 3) - 550 * eta * bhp) - math.sqrt(k1)

def test_example_19_10():
    weight = 3400 # lbs
    altitude = 14_000 # ft
    power_set = 0.55 # 55% power
    cruising_speed = 169 # KTAS at 14,000 ft with 55% power
    cd_min = 0.02541 # example 15.18
    oswald_efficiency = 0.7566 # example 15.18
    wing_area = 144.9 # ft^2
    density = 0.001546 # slugs/ft^3 at 14,000 ft
    eta = 0.85 # propeller efficiency
    bhp = 310 # bhp max-rated sea-level power

    # $p S C_{Dmin}$
    p_s_cd_min = density * wing_area * cd_min
    display(f"{p_s_cd_min=}")

    # $$550 n_p P_{BHP}$
    propeller_power = 550 * eta * (power_set * bhp)
    display(f"{propeller_power=}")

    weight_drag = math.pow(4 * weight, 2) * cd_min * oswald_efficiency
    display(f"{weight_drag=}")

    # the cruising speed at 55%
    v_max = eq19_2_14_max_level_airspeed_prop(
        wing_area,
        oswald_efficiency,
        cd_min,
        weight,
        altitude,
        bhp,
        eta,
    )
    return v_max

    # 283.4 ft/s, 167.9 KTAS
test_example_19_10()

'p_s_cd_min=0.005692231314'

'propeller_power=79708.75'

'weight_drag=3555894.10176'

499.9999701976776